In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix,accuracy_score, recall_score, precision_score, f1_score,classification_report 
from sklearn.metrics import roc_curve, auc

import model_team14 
from model_team14 import *

In [2]:
def make_auc_dict(model_name, clf, labels, y_test, y_pred, y_pred_prob):

    dict_auc={'model':[], 'class':[], 'auc':[]}

    # 1이면, 1/2이면 각각의 auc score

    if len(labels)==2:

        #class_probabilities = model.predict_proba(X_test)
#        preds = y_pred_prob[:, 1]

        fpr, tpr, threshold = roc_curve(y_test, y_pred)
        roc_auc = auc(fpr, tpr)

        dict_auc['model'].append(model_name)
        dict_auc['class'].append(0)
        dict_auc['auc'].append(roc_auc)

        dict_auc['model'].append(model_name)
        dict_auc['class'].append(1)
        dict_auc['auc'].append(roc_auc)

    else:
        #def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
       ## y_score = ### model.predict_proba(X_test)

        #Binarize the output
        y_test_bin = label_binarize(y_test, classes=sorted(labels))
#        print(y_test_bin)
        n_classes = y_test_bin.shape[1]

        fpr = dict()
        tpr = dict()
        roc_auc = dict()
                                    
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])  #y_score[:, i]

#             if 'lstm' in model_name:
#                 fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i].detach().numpy())  #y_score[:, i]
#             else:
#                 fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])  #y_score[:, i]
            dict_auc['model'].append(model_name)
            dict_auc['class'].append(i)
            dict_auc['auc'].append(auc(fpr[i], tpr[i]))

    return pd.DataFrame(dict_auc)

In [3]:
def make_otherreport():

    folders=[x for x in os.listdir("../result") if ('ft' in x)]  ##&('ls' not in x)

    for i, folder in enumerate(folders):
    #    print(folder)
        reports=[x for x in os.listdir("../result/{}".format(folder)) if 'creport' in x]
#        print(len(reports), reports)
    
        models=[x for x in os.listdir("../result/{}".format(folder)) if 'clf' in x]
#        print(len(models), models)
        
        ft_criteria=folder.split('_')[1]
        ft_threshold=folder.split('_')[2]
        
        ft_scaling=folder.split('_')[-1]
        if ft_scaling=='scaling':
            ft_scaling='scaled'
        else:
            ft_scaling='non-scaled'

        path='../result/{}'.format(folder)

        for k, report in enumerate(reports):
            with open (path+'/{}'.format(report), 'rb') as f:
                df=pickle.load(f)
            if k==0:
                df_report=df
            else:    
                df_report=pd.concat([df_report, df])

        df_report['ft_criteria']=ft_criteria    
        df_report['ft_threshold']=ft_threshold
        df_report['ft_scaling']=ft_scaling
        
         
        print('folder is', folder)

        for j, model in enumerate(models):
    #        print(model)
            model_name=model.split('.')[0]
            dtype=model.split('_')[4]
            y_type=model.split('_')[2]+'_'+model.split('_')[3]
            test_year=5

            with open (path+'/{}'.format(model), 'rb') as f:
                [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)

            if dtype=='tr':
                X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
                y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
            else:
                X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
                y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)

            labels=list(y[y_type].unique())    
            y_test=y[y_type][-(test_year*12):]

##        print(labels, y_type)

            df_tmp=make_auc_dict(model_name, clf, labels, y_test, y_pred, y_pred_prob)
    ##        print('auc df completed')
    ##        print(df_tmp.head())

            if j==0:
                df_auc=df_tmp
            else:
                df_auc=pd.concat([df_auc, df_tmp])

        df_report=df_report.merge(df_auc, left_on=['class','model'], right_on=['class','model'])

        if i==0:
            df_final_report=df_report
        else:
            df_final_report=pd.concat([df_final_report, df_report])
            
    return df_final_report            

In [4]:
df_creport_other=make_otherreport()
df_creport_other.head()
len(df_creport_other)

folder is ft_None_0.2
folder is ft_None_0.2_scaling
folder is ft_None_0.3
folder is ft_None_0.3_scaling
folder is ft_None_0.4
folder is ft_None_0.4_scaling
folder is ft_None_0.5
folder is ft_None_0.5_scaling


480

In [5]:
df_selection=df_creport_other.groupby(['ft_criteria','ft_threshold','ft_scaling','model']).mean().reset_index()
df_selection.head()

,ft_criteria,ft_threshold,ft_scaling,model,class,precision,recall,f1_score,support,accuracy,auc
0,None,0.2,non-scaled,clf_knn_y_agg_ntr_tss_t5_spl5,1.0,0.316667,0.333333,0.323333,20.0,0.93,0.538537
1,None,0.2,non-scaled,clf_knn_y_agg_ntr_tss_t5_spl5_km,1.0,0.310000,0.333333,0.323333,20.0,0.93,0.500000
2,None,0.2,non-scaled,clf_knn_y_agg_tr_tss_t5_spl5,1.0,0.376667,0.590000,0.370000,20.0,0.75,0.918514
3,None,0.2,non-scaled,clf_knn_y_agg_tr_tss_t5_spl5_km,1.0,0.456667,0.566667,0.460000,20.0,0.68,0.689347
4,None,0.2,non-scaled,clf_knn_y_oecd_ntr_tss_t5_spl5,0.5,0.465000,0.500000,0.485000,30.0,0.93,0.500000


In [6]:
final_selection=df_selection.sort_values(['recall','accuracy'], ascending=[False, False])

with open('../result/final_selection.pkl','wb') as f:
    pickle.dump(final_selection, f)

In [17]:
final_selection[:20]  ##[final_selection.ft_scaling=='non-scaled'][:10]

,ft_criteria,ft_threshold,ft_scaling,model,class,precision,recall,f1_score,support,accuracy,auc
34,None,0.2,scaled,clf_rf_y_agg_tr_tss_t5_spl5,1.0,0.566667,0.940000,0.633333,20.0,0.83,0.955229
54,None,0.3,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.655000,0.920000,0.690000,30.0,0.85,0.919643
6,None,0.2,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885000,0.625000,30.0,0.78,0.883929
102,None,0.4,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885000,0.625000,30.0,0.78,0.883929
150,None,0.5,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885000,0.625000,30.0,0.78,0.883929
12,None,0.2,non-scaled,clf_rf_y_oecd_ntr_tss_t5_spl5,0.5,0.990000,0.875000,0.925000,30.0,0.98,0.875000
36,None,0.2,scaled,clf_rf_y_oecd_ntr_tss_t5_spl5,0.5,0.990000,0.875000,0.925000,30.0,0.98,0.875000
18,None,0.2,non-scaled,clf_svc_y_agg_tr_tss_t5_spl5,1.0,0.690000,0.840000,0.603333,20.0,0.55,0.874897
76,None,0.3,scaled,clf_knn_y_oecd_ntr_tss_t5_spl5,0.5,0.680000,0.830000,0.720000,30.0,0.90,0.830357
124,None,0.4,scaled,clf_knn_y_oecd_ntr_tss_t5_spl5,0.5,0.680000,0.830000,0.720000,30.0,0.90,0.830357


In [8]:
# with open('../result/ft_None_{}{}/df_{}.pkl'.format(0.2, '_scaling', 'rf'),'rb') as f:
#         df_tmp=pickle.load(f)

# df_tmp.sort_values('mean_test_recall', ascending=False)

In [9]:
## Let's see the result of cross validation!

dict_cv={}
thresholds=[0.2,0.3,0.4,0.5]
samplings=['_scaling',''] # ['_scaling','']

for sampling in samplings:
    for threshold in thresholds:
        for i, model in enumerate(['knn','rf','svc','knn_km','rf_km','svc_km']):
            with open('../result/ft_None_{}{}/df_{}.pkl'.format(threshold, sampling, model),'rb') as f:
                df_tmp=pickle.load(f)
                dict_cv['{}_{}'.format(sampling, threshold)+model]=dict(df_tmp.sort_values(['mean_test_recall'], ascending=[False]).iloc[0])
        
final_cv=pd.DataFrame(dict_cv).T.sort_values('mean_test_recall', ascending=False).reset_index().rename(columns={'index':'hyperparam'})

with open('../result/final_cv.pkl','wb') as f:
    pickle.dump(final_cv, f)

In [35]:
thresholds=[0.2,0.3,0.4,0.5]
scalings=['_scaling',''] # ['_scaling','']

df_cv=pd.DataFrame()

for scaling in scalings:
    for threshold in thresholds:
        for i, model in enumerate(['knn','rf','svc','knn_km','rf_km','svc_km']):
            with open('../result/ft_None_{}{}/df_{}.pkl'.format(threshold, sampling, model),'rb') as f:
                df_tmp=pickle.load(f)
                df_tmp['threshold']=threshold
                df_tmp['scaling']=scaling
            df_cv=pd.concat([df_cv, df_tmp])

In [43]:
df_cv[(df_cv.threshold==0.3)
      &(df_cv.y=='y_oecd')
      &(df_cv.data=='tr')
      &(df_cv.scaling=='_scaling')      
      &(df_cv.rank_test_recall==1)            
     ].sort_values('mean_test_recall', ascending=False)[:10]

,n_neighbors,weights,model,data,y,cv,mean_test_recall,std_test_recall,rank_test_recall,threshold,scaling,class_weight,max_features,n_estimators,warm_start,C,kernel
6,NaN,NaN,RF,tr,y_oecd,tss,0.746513,0.063187,1,0.3,_scaling,balanced,0.3,30.0,True,NaN,NaN
7,NaN,NaN,RF,tr,y_oecd,tss,0.746513,0.063187,1,0.3,_scaling,balanced,0.3,30.0,False,NaN,NaN
8,10.0,uniform,KNN,tr,y_oecd,tss,0.703995,0.128376,1,0.3,_scaling,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,SVC,tr,y_oecd,tss,0.693390,0.065548,1,0.3,_scaling,NaN,NaN,NaN,NaN,2.0,rbf
32,NaN,NaN,RF,tr,y_oecd,tss,0.667786,0.057932,1,0.3,_scaling,balanced_subsample,0.2,50.0,True,NaN,NaN
33,NaN,NaN,RF,tr,y_oecd,tss,0.667786,0.057932,1,0.3,_scaling,balanced_subsample,0.2,50.0,False,NaN,NaN
38,NaN,NaN,RF,tr,y_oecd,tss,0.667786,0.057932,1,0.3,_scaling,balanced_subsample,0.3,50.0,True,NaN,NaN
39,NaN,NaN,RF,tr,y_oecd,tss,0.667786,0.057932,1,0.3,_scaling,balanced_subsample,0.3,50.0,False,NaN,NaN
8,10.0,uniform,KNN,tr,y_oecd,tss,0.657454,0.039490,1,0.3,_scaling,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,SVC,tr,y_oecd,tss,0.649191,0.107472,1,0.3,_scaling,NaN,NaN,NaN,NaN,2.0,sigmoid


In [20]:
final_cv[(final_cv.hyperparam.str.contains('scaling'))&
         (final_cv.hyperparam.str.contains('0.2'))&
         (final_cv.y.str.contains('agg'))]

,hyperparam,n_neighbors,weights,model,data,y,cv,mean_test_recall,std_test_recall,rank_test_recall,class_weight,max_features,n_estimators,warm_start,C,kernel


In [12]:
final_cv[(final_cv.hyperparam.str.contains('scaling'))]

,hyperparam,n_neighbors,weights,model,data,y,cv,mean_test_recall,std_test_recall,rank_test_recall,class_weight,max_features,n_estimators,warm_start,C,kernel
2,_scaling_0.5rf,NaN,NaN,RF,tr,y_oecd,tss,0.761198,0.061318,1,None,0.7,30,False,NaN,NaN
5,_scaling_0.4rf,NaN,NaN,RF,tr,y_oecd,tss,0.758836,0.048309,1,None,0.2,30,False,NaN,NaN
8,_scaling_0.3rf,NaN,NaN,RF,tr,y_oecd,tss,0.746513,0.063187,1,balanced,0.3,30,True,NaN,NaN
10,_scaling_0.4knn,5,uniform,KNN,tr,y_oecd,tss,0.740769,0.058381,1,NaN,NaN,NaN,NaN,NaN,NaN
11,_scaling_0.5knn,5,distance,KNN,tr,y_oecd,tss,0.735817,0.079077,1,NaN,NaN,NaN,NaN,NaN,NaN
12,_scaling_0.3knn,10,uniform,KNN,tr,y_oecd,tss,0.703995,0.128376,1,NaN,NaN,NaN,NaN,NaN,NaN
13,_scaling_0.3svc,NaN,NaN,SVC,tr,y_oecd,tss,0.69339,0.065548,1,NaN,NaN,NaN,NaN,2.0,rbf
14,_scaling_0.2rf,NaN,NaN,RF,tr,y_oecd,tss,0.687388,0.10887,1,balanced,0.5,50,True,NaN,NaN
17,_scaling_0.2svc,NaN,NaN,SVC,tr,y_oecd,tss,0.676671,0.080999,1,NaN,NaN,NaN,NaN,2.0,sigmoid
19,_scaling_0.5svc,NaN,NaN,SVC,ntr,y_oecd,tss,0.669462,0.070221,1,NaN,NaN,NaN,NaN,1.0,sigmoid


In [13]:
# with open('../result/ft_None_0.2_scaling')

# df_creport_other['model_type']=df_creport_other['model'].apply(lambda x:x.split('_')[1])
# df_creport_other['X_type']=df_creport_other['model'].apply(lambda x:x.split('_')[4])

# df_selection=df_creport_other.groupby(['ft_criteria','ft_threshold','model']).mean(['recall',
#                                                                        'precision',
#                                                                        'auc',
#                                                                       'fl_score']).reset_index()

# df_selection.sort_values(['auc','recall'], ascending=[False,False])[:20]

In [14]:
# df_others_report=df_report.groupby(['ft_criteria','ft_threshold','model']).mean().reset_index().drop(columns=['class','f1_score','support'])

In [15]:
# with open('../result/df_lstm_creport.pkl', 'rb') as f:
#     df_lstm_report=pickle.load(f)

# df_lstm_report.sort_values(['auc'], ascending=[False])

In [16]:
# with open ('../result/df_final_report.pkl', 'wb') as f:
#     pickle.dump(df_final_report, f)
    
# with open ('../result/df_ls_report.pkl', 'wb') as f:
#     pickle.dump(df_creport_ls, f)

    
# df_final_report.head()

NameError: name 'df_final_report' is not defined

In [45]:
final_selection
final_selection['model_type']=final_selection['model'].apply(lambda x:x.split('_')[1])
final_selection['y_type']=final_selection['class'].apply(lambda x:'binary' if x==0.5 else 'trinary')
final_selection['dtype']=final_selection['model'].apply(lambda x:'non-stationary' if x.split('_')[4]=='ntr' else 'stationary')

final_selection.head(10)

,ft_criteria,ft_threshold,ft_scaling,model,class,precision,recall,f1_score,support,accuracy,auc,model_type,y_type,dtype
34,None,0.2,scaled,clf_rf_y_agg_tr_tss_t5_spl5,1.0,0.566667,0.940,0.633333,20.0,0.83,0.955229,rf,trinary,stationary
54,None,0.3,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.655000,0.920,0.690000,30.0,0.85,0.919643,knn,binary,stationary
6,None,0.2,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885,0.625000,30.0,0.78,0.883929,knn,binary,stationary
102,None,0.4,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885,0.625000,30.0,0.78,0.883929,knn,binary,stationary
150,None,0.5,non-scaled,clf_knn_y_oecd_tr_tss_t5_spl5,0.5,0.620000,0.885,0.625000,30.0,0.78,0.883929,knn,binary,stationary
12,None,0.2,non-scaled,clf_rf_y_oecd_ntr_tss_t5_spl5,0.5,0.990000,0.875,0.925000,30.0,0.98,0.875000,rf,binary,non-stationary
36,None,0.2,scaled,clf_rf_y_oecd_ntr_tss_t5_spl5,0.5,0.990000,0.875,0.925000,30.0,0.98,0.875000,rf,binary,non-stationary
18,None,0.2,non-scaled,clf_svc_y_agg_tr_tss_t5_spl5,1.0,0.690000,0.840,0.603333,20.0,0.55,0.874897,svc,trinary,stationary
76,None,0.3,scaled,clf_knn_y_oecd_ntr_tss_t5_spl5,0.5,0.680000,0.830,0.720000,30.0,0.90,0.830357,knn,binary,non-stationary
124,None,0.4,scaled,clf_knn_y_oecd_ntr_tss_t5_spl5,0.5,0.680000,0.830,0.720000,30.0,0.90,0.830357,knn,binary,non-stationary


In [ ]:
# df_selection=df_final_report.groupby(['ft_criteria','ft_threshold','model']  ##[df_final_report['class']>0]
#                                                    ).mean().reset_index()

# #df_creport_ls[df_creport_ls['class']==1].sort_values('recall', ascending=False)

In [ ]:
# df_selection=df_selection.sort_values(['recall','auc'], ascending=[False, False]).reset_index()
# df_selection['model_type']=df_selection['model'].apply(lambda x:x.split('_')[1])
# df_selection['y_type']=df_selection['class'].apply(lambda x:'binary' if x==0.5 else 'trinary')
# df_selection['dtype']=df_selection['model'].apply(lambda x:'non-stationary' if x.split('_')[4]=='ntr' else 'stationary')


# df_selection.head(10)

In [ ]:
# with open ('../result/ft_None_0.2/clf_rf_y_agg_tr_block_t5_spl3.pkl','rb') as f:
#     [clf_rf,_,_,_,_]=pickle.load(f)
#     best_rf_model=clf_rf.best_estimator_
    
# with open ('../result/ft_None_0.4/clf_knn_y_oecd_tr_block_t5_spl3.pkl','rb') as f:
#     [clf_knn,_,_,_,_]=pickle.load(f)
#     best_knn_model=clf_knn.best_estimator_

# best_lstm_model=load_model('../result/ft_None_0.2_ls/clf_lstm_y_oecd_24_b5_ep300_h100.h5', compile=False)
# best_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC','Accuracy','Recall', 'Precision'])


In [ ]:
# best_lstm_model=load_model('../result/ft_None_0.2_ls/clf_lstm_y_oecd_24_b5_ep300_h100.h5', compile=True)

In [ ]:
# best_lstm_model.

# _validate_and_get_metrics_result()

In [ ]:
# dir(best_lstm_model)

In [ ]:
# # dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_class_weight', 'param_max_features', 'param_n_estimators', 'param_warm_start', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])
# np.argmin(clf_rf.cv_results_['mean_test_score'])
# clf_rf.cv_results_['mean_test_score'][94]
# clf_rf.cv_results_['std_test_score'][94]
# clf_rf.cv_results_['params'][94]

In [ ]:
# best_rf_model

In [ ]:
# best_knn_model

In [ ]:
# best_lstm_model.summary()

In [46]:
import altair as alt

base1=alt.Chart(final_selection).mark_point(size=30, shape='circle').encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.2, domainMax=1),  ##domainMin=0.45, domainMax=0.7
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.1, domainMax=1),  ##domainMin=0.75, domainMax=0.9
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
    color=alt.Color('y_type:N', scale=alt.Scale(range=['blue','red']), title='# of classes'), #scheme= 'turbo',domainMin=0.78, domainMax=0.86
    shape=alt.Shape('model_type:N', title='model type')
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', ), title='auc score') #domainMin=0.78, domainMax=0.86
#).transform_filter(alt.datum.model_type=='knn')
).transform_filter((alt.datum.precision>0.2)&(alt.datum.recall>0.1))


emp1=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==74)


emp2=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==12)

emp3=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==102)



(base1+emp1+emp2+emp3).properties(                       ##text0+text1+text2
    width=200, height=300,
    title={'text':['Precision(X) and Recall(Y) of candidate models'],
       'subtitle':['the annotated point: The Best Model for each model type'],
       'align':'left',
       'anchor':'start'
      })

alt.LayerChart(...)

In [47]:
import altair as alt

base1=alt.Chart(final_selection).mark_point(size=30, shape='circle').encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),  ##domainMin=0.45, domainMax=0.7
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),  ##domainMin=0.75, domainMax=0.9
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
    color=alt.Color('dtype:N', scale=alt.Scale(range=['blue','red']), title='# of classes'), #scheme= 'turbo',domainMin=0.78, domainMax=0.86
    shape=alt.Shape('model_type:N', title='model type')
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', ), title='auc score') #domainMin=0.78, domainMax=0.86
#).transform_filter(alt.datum.model_type=='knn')
)


emp1=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==74)


emp2=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==12)

emp3=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==102)



(base1+emp1+emp2+emp3).properties(                       ##text0+text1+text2
    width=400, height=300,
    title={'text':['Precision(X) and Recall(Y) of candidate models'],
       'subtitle':['the annotated point: The Best Model for each model type'],
       'align':'left',
       'anchor':'start'
      })

alt.LayerChart(...)

In [48]:
import altair as alt

base1=alt.Chart(final_selection).mark_point(size=30, shape='circle').encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=0.85),  ##domainMin=0.45, domainMax=0.7
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),  ##domainMin=0.75, domainMax=0.9
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
    color=alt.Color('dtype:N', scale=alt.Scale(range=['blue','red']), title='# of classes'), #scheme= 'turbo',domainMin=0.78, domainMax=0.86
    shape=alt.Shape('model_type:N', title='model type')
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', ), title='auc score') #domainMin=0.78, domainMax=0.86
#).transform_filter(alt.datum.model_type=='knn')
).transform_filter((alt.datum.precision>0.5)&(alt.datum.precision<0.85)&(alt.datum.recall>0.5))


emp1=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==74)


emp2=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==12)

emp3=alt.Chart(final_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
).transform_filter(alt.datum.index==102)



(base1+emp1+emp2+emp3).properties(                       ##text0+text1+text2
    width=200, height=300,
    title={'text':['Precision(X) and Recall(Y) of candidate models'],
       'subtitle':['the annotated point: The Best Model for each model type'],
       'align':'left',
       'anchor':'start'
      })

alt.LayerChart(...)

In [49]:
import altair as alt

base1=alt.Chart(final_selection).mark_point(size=30, shape='circle').encode(
    x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=0.75),  ##domainMin=0.45, domainMax=0.7
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),  ##domainMin=0.75, domainMax=0.9
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
    color=alt.Color('ft_threshold:Q', scale=alt.Scale(scheme='redblue'), title='threshold'), #scheme= 'turbo',domainMin=0.78, domainMax=0, range=['darkblue','cyan','red'].86
#    shape=alt.Shape('model_type:N', title='model type')
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', ), title='auc score') #domainMin=0.78, domainMax=0.86
#).transform_filter(alt.datum.model_type=='knn')
).transform_filter((alt.datum.precision>0.5)&(alt.datum.precision<0.75)&(alt.datum.recall>0.5)&(alt.datum.model_type=='lstm'))


# emp1=alt.Chart(df_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
# #    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
# ).transform_filter(alt.datum.index==74)


# emp2=alt.Chart(df_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
# #    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
# ).transform_filter(alt.datum.index==12)

# emp3=alt.Chart(df_selection).mark_circle(size=120, filled=False, stroke='orange', strokeWidth=5).encode(
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0.5, domainMax=1),
#            title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
# #    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', domainMin=0.78, domainMax=0.86), title='auc score')
# ).transform_filter(alt.datum.index==102)



(base1).properties(                       ## +emp1+emp2+emp3 ##text0+text1+text2
    width=200, height=300,
    title={'text':['Precision(X) and Recall(Y) of lstm models'],
       'subtitle':['the annotated point: The Best Model for each model type'],
       'align':'left',
       'anchor':'start'
      })

alt.Chart(...)

In [ ]:
# text0=alt.Chart(df_selection).mark_text(
#     text='The Best RF Model',
#     color='black',
#     align='left',
#     baseline='middle',
#     dx=-50, dy=-10).encode(  # dx=-5, dy=-10
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),  ##domainMin=0.45, domainMax=0.7
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),   ##domainMin=0.75, domainMax=0.9
#            title='How completely cycle can be detected?'), #domainMin    
#   ).transform_filter(alt.datum.index==74)


# text1=alt.Chart(df_selection).mark_text(
#     text='The Best LSTM Model',
#     color='black',
#     align='left',
#     baseline='middle',
#     dx=-50, dy=-10).encode(  # dx=-5, dy=-10
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),  ##domainMin=0.45, domainMax=0.7
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),   ##domainMin=0.75, domainMax=0.9
#            title='How completely cycle can be detected?'), #domainMin    
#   ).transform_filter(alt.datum.index==12)


# text2=alt.Chart(df_selection).mark_text(
#     text='The Best HNN Model',
#     color='black',
#     align='left',
#     baseline='middle',
#     dx=-50, dy=-10).encode(  # dx=-5, dy=-10
#     x=alt.X('precision:Q', scale=alt.Scale(domainMin=0, domainMax=1),  ##domainMin=0.45, domainMax=0.7
#            title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
#     y=alt.Y('recall:Q', scale=alt.Scale(domainMin=0, domainMax=1),   ##domainMin=0.75, domainMax=0.9
#            title='How completely cycle can be detected?'), #domainMin    
#   ).transform_filter(alt.datum.index==102)

In [50]:
import altair as alt

base1=alt.Chart(final_selection, width=300, height=300).mark_point(shape='circle').encode(
    x=alt.X('precision:Q', scale=alt.Scale(),  ##domainMin=0.45, domainMax=0.7
           title='Is classified cycle true?'), #domainMin=0.0, domainMax=0.37
    y=alt.Y('recall:Q', scale=alt.Scale(),  ##domainMin=0.75, domainMax=0.9
           title='How completely cycle can be detected?'), #domainMin=0.9, domainMax=1.0
    color=alt.Color('class:Q', scale=alt.Scale(scheme='turbo'), title='class'), #scheme= 'turbo',domainMin=0.78, domainMax=0.86
    shape=alt.Shape('model_type:N')
#    color=alt.Color('auc:Q', scale=alt.Scale(scheme= 'turbo', ), title='auc score') #domainMin=0.78, domainMax=0.86
) #.transform_filter(alt.datum.model_type=='knn')

base1

alt.Chart(...)